In [37]:
pip install scipy

You should consider upgrading via the '/Users/anaistang/.virtualenvs/web/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import requests 
import pandas as pd 
import requests 
import math 
from scipy.stats import percentileofscore as score
import xlsxwriter

In [40]:
from starter.secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('starter/sp_500_stocks.csv')


## Getting one-year price returns for each stock in the universe

In [5]:
symbol = 'aapl'
url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
r = requests.get(url)
data = r.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2259997339949,
 'week52high': 151.41,
 'week52low': 88.3,
 'week52highSplitAdjustOnly': 150.4,
 'week52lowSplitAdjustOnly': 89,
 'week52change': 0.5083248069430318,
 'sharesOutstanding': 17322928475,
 'float': 0,
 'avg10Volume': 83648384,
 'avg30Volume': 76372161,
 'day200MovingAvg': 133.1,
 'day50MovingAvg': 130.52,
 'employees': 149515,
 'ttmEPS': 4.64,
 'ttmDividendRate': 0.854889345932639,
 'dividendYield': 0.006437005696199516,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-30',
 'nextEarningsDate': '2021-07-15',
 'peRatio': 29.809175981594365,
 'beta': 1.55591893680141,
 'maxChangePercent': 49.61249743608207,
 'year5ChangePercent': 5.065695169097023,
 'year2ChangePercent': 1.73752821140607,
 'year1ChangePercent': 0.4977837070870686,
 'ytdChangePercent': -0.013771508361123437,
 'month6ChangePercent': 0.03467239533035248,
 'month3ChangePercent': 0.08695595676480292,
 'month1ChangePercent': 0.04556023561983153,
 'day30ChangePercent': 

## Parsing API call


In [6]:
change_percent = data['year1ChangePercent']
change_percent

0.4977837070870686

In [7]:
def chunks(lst, n):
    """
    Splitting list into evenly sized chunks
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]



In [8]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [9]:
columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data['A'])
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
            [
                symbol, 
                data[symbol]['stats']['week52high'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
            ],
            index=columns),
            ignore_index=True
            )
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,150.96,0.671627,N/A
1,AAL,26.21,0.364777,N/A
2,AAP,212.95,0.327811,N/A
3,AAPL,145.14,0.502128,N/A
4,ABBV,119.27,0.242266,N/A
...,...,...,...,...
95,CINF,123.90,0.851633,N/A
96,CL,89.13,0.131360,N/A
97,CLX,241.00,-0.177825,N/A
98,CMA,81.31,0.787049,N/A


## Removing Low-Momentum Stocks
- We're identifying 50 highest-momentum stocks in the S&P 500
- We need to remove all the stocks in our DataFrame that fall below this momentum threshold.
- Sort the Dataframe by the stocks one-year price return, drop all stocks outside the top 50.

In [10]:
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True) # sets first row as first index 
final_df 

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,668.20,1.337336,N/A
1,29,AMAT,149.63,1.216316,N/A
2,78,CARR,48.92,1.181510,N/A
3,23,ALB,192.18,1.041956,N/A
4,45,APTV,166.47,0.948072,N/A
5,16,AES,29.52,0.937285,N/A
6,95,CINF,123.90,0.851633,N/A
7,82,CBRE,91.92,0.823169,N/A
8,51,AVY,231.12,0.795635,N/A
9,98,CMA,81.31,0.787049,N/A


In [11]:
## Calculating the number of shares to buy 

In [63]:
def portfolio_size():
    global portfolio_size 
    portfolio_size = input('Enter the size of your portfolio:\n')
    
    try:
        float(portfolio_size)
    except:
        print('This is not a number \n Please try again:')
        portfolio_size = input('Enter the size of your portfolio')
        
portfolio_size()
print(portfolio_size)
        

Enter the size of your portfolio:
1000000000
1000000000


In [64]:
position_size = float(portfolio_size)/len(final_df.index)
# print(position_size)
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
final_df

/Users/anaistang/.virtualenvs/web/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,668.20,1.337336,29931
1,29,AMAT,149.63,1.216316,133663
2,78,CARR,48.92,1.181510,408830
3,23,ALB,192.18,1.041956,104069
4,45,APTV,166.47,0.948072,120141
5,16,AES,29.52,0.937285,677506
6,95,CINF,123.90,0.851633,161420
7,82,CBRE,91.92,0.823169,217580
8,51,AVY,231.12,0.795635,86535
9,98,CMA,81.31,0.787049,245972


## Building a realistic momentum strategy 
- High-quality momentum stocks show 'slow and steady' outperformance over long periods of time 
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards

High-quality momentum stocks are preferred because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as FDA approval for a biotech company)

To identify high-quality momentum, building the following strategies will help select stocks from highest percentiles of
1. 1-month price returns 
2. 3-month price returns 
3. 6-month price returns 
4. 1-year price returns 


In [58]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-year price return',
    'One-year return percentile',
    'Six-month price return',
    'Six-month return percentile',
    'Three-month price return',
    'Three-month return percentile',
    'One-month price return',
    'One-month return percentile'
]

hqm_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
#         print(data)
        hqm_df = hqm_df.append(
            
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A'
                
            ],
            index = hqm_columns),
            ignore_index=True
        )

hqm_df.columns
    

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-year price return',
       'One-year return percentile', 'Six-month price return',
       'Six-month return percentile', 'Three-month price return',
       'Three-month return percentile', 'One-month price return',
       'One-month return percentile'],
      dtype='object')

## Calculating Momentum Percentiles 

- Calculate momentum percentile scores for every stock in the universe
- We need to calculate percentile scores for the following metrics for every stock:

    - `One-Year price return`
    - `Six month price return`
    - `Three-month price return`
    - `One-month price return`


In [62]:
time_periods = [
                'One-year',
                'Six-month',
                'Three-month',
                'One-month'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} return percentile'] = stats.percentileofscore(hqm_df[f'{time_period} price return'], hqm_df.loc[row, f'{time_period} price return'])/100

for time_period in time_periods:
    print(hqm_df[f'{time_period} return percentile'])

hqm_df
                                                                                                                                                                  


TypeError: '<' not supported between instances of 'NoneType' and 'float'